From Jerry Meyers, a careening commute problem:

Four co-workers carpool to work each day. A driver is selected randomly for the drive to work and again randomly for the drive home. Each of the drivers has a lead foot, and each has a chance of being ticketed for speeding. Driver A has a 10 percent chance of getting a ticket each time he drives, Driver B a 15 percent chance, Driver C a 20 percent chance, and Driver D a 25 percent chance. The state will immediately revoke the license of a driver after his or her third ticket, and a driver will stop driving in the carpool once his license is revoked. Since there is only one police officer on the carpool rout, a maximum of one ticket will be issued per morning and a max of one per evening.

Assuming that all four drivers start with no tickets, how many days can we expect the carpool to last until all the drivers have lost their licenses?

In [202]:
import numpy as np
import itertools

In [203]:
def prob_transition(initial, final, ticket_limit=3):
    initial = np.array(initial)
    final = np.array(final)
    # only 1 driver ticket has increased by exactly 1 ticket
    ticket_diff = np.subtract(final, initial)
    if len(np.argwhere(ticket_diff < 0)) > 0 \
    or len(np.argwhere(ticket_diff > 1)) > 0 \
    or len(np.argwhere(ticket_diff == 1)) > 1 \
    or len(np.argwhere(ticket_diff == 1)) == 0:
        # can't remove tickets, invalid transition
        return 0
    
    tickets_issued = np.count_nonzero(ticket_diff)
    # 1 ticket issued, calc prob
    avail_drivers = len(np.argwhere(initial < ticket_limit))
    ticketed_driver_indx = np.argwhere(ticket_diff == 1)[0][0]
        
    return (1/avail_drivers) * probs_pulled_over[ticketed_driver_indx]

In [204]:
TICKETS_TO_SUSPENSION = 3
probs_pulled_over = np.array([0.10, 0.15, 0.20, 0.25])
NUM_DRIVERS = len(probs_pulled_over)

In [205]:
states = {i:state for i,state in enumerate(itertools.product(range(0,TICKETS_TO_SUSPENSION+1), repeat = NUM_DRIVERS))}
num_states = len(states.keys())

In [206]:
prob_trans_arr = np.zeros((num_states,num_states))

In [207]:
for row in range(0,num_states):
    for col in range(row+1, num_states):
        prob_trans_arr[row][col] = prob_transition(states[row], states[col], TICKETS_TO_SUSPENSION)

In [208]:
for row,col in zip(*np.diag_indices_from(prob_trans_arr)):
    prob_trans_arr[row][col] = 1 - np.sum(prob_trans_arr[row])
    

In [209]:
prob_trans_arr = np.delete(prob_trans_arr, -1, 0)
prob_trans_arr = np.delete(prob_trans_arr, -1, 1)

In [210]:
q = np.mat(prob_trans_arr)
i = np.identity(num_states - 1)
n = np.linalg.inv(i - q)
expected_trips_to_states = np.array(n * np.ones((num_states-1,1)))
expected_trips_to_all_suspended = expected_trips_to_states[0][0]
expected_days_to_all_suspended = expected_trips_to_all_suspended / 2
expected_days_to_all_suspended

38.499999999999957